### Test Pyclick

In [1]:
pwd

'C:\\Users\\gauthieca\\Desktop\\Code_ Thèse\\bandits-to-rank\\Test'

In [2]:
cd ../../

C:\Users\gauthieca\Desktop\Code_ Thèse


In [3]:
from __future__ import print_function
import sys 
import time 
from pyclick.click_models.Evaluation import LogLikelihood, Perplexity
from pyclick.click_models.UBM import UBM
from pyclick.click_models.DBN import DBN
from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.DCM import DCM
from pyclick.click_models.CCM import CCM
from pyclick.click_models.CTR import DCTR, RCTR, GCTR
from pyclick.click_models.CM import CM
from pyclick.click_models.PBM import PBM
from pyclick.utils.Utils import Utils
from pyclick.utils.YandexRelPredChallengeParser import YandexRelPredChallengeParser

from pyclick.search_session.SearchResult import SearchResult
from pyclick.click_models.task_centric.TaskCentricSearchSession import TaskCentricSearchSession

from random import sample
from random import random

In [4]:
click_model = PBM()
search_sessions_path = 'PyClick-master/PyClick-master/examples/data/YandexPersonalizedChallenge'
search_sessions_num = 24615 

search_sessions = YandexRelPredChallengeParser().parse(search_sessions_path, search_sessions_num)


In [5]:
search_sessions[0]

{"query": "0", "task": "34573630", "web_results": [{"click": 0, "id": "3140263,2771769,3809197"}, {"click": 0, "id": "34175267,3279130"}, {"click": 0, "id": "34171511,3279018"}, {"click": 0, "id": "35444452,3339757"}, {"click": 0, "id": "15370141,1583351"}, {"click": 0, "id": "31342884,3075887"}, {"click": 0, "id": "43630531,3823198"}, {"click": 0, "id": "26065978,2597528"}, {"click": 0, "id": "29902424,2902450"}, {"click": 0, "id": "39016998,3549467"}, {"click": 0, "id": "62861215,4825939"}]}

In [6]:
liste=search_sessions[0].web_results[1].id.split(',')
dicti={}
for pos in range(len(liste)):
    dicti[liste[pos]]={}
    dicti[liste[pos]][pos]=1
print(dicti)

{'34175267': {0: 1}, '3279130': {1: 1}}


In [7]:
def get_item(session,shown_for_query,dict_len_prop):
    web_results = session.web_results
    for web_res in web_results :
        #print (web_res.id)
        liste_item_shown=web_res.id.split(',')
        nb_prop=len(liste_item_shown)
        if nb_prop in dict_len_prop.keys():
            dict_len_prop[nb_prop]+=1
        else :
            dict_len_prop[nb_prop]=1
        for pos in range(len(liste_item_shown)):
            item = liste_item_shown[pos]
            if item not in shown_for_query.keys() :
                dicti[item]={}
                dicti[item][pos]=1
            else :
                if pos not in dicti[item].keys():
                    dicti[item][pos]=1
                else : 
                    dicti[item][pos]+=1
    return dicti

In [16]:
def get_item_if_2pos(session,shown_for_query,dict_len_prop):
    web_results = session.web_results
    for web_res in web_results :
        #print (web_res.id)
        liste_item_shown=web_res.id.split(',')
        nb_prop=len(liste_item_shown)
        if nb_prop <= 2:
            if nb_prop in dict_len_prop.keys():
                dict_len_prop[nb_prop]+=1
            else :
                dict_len_prop[nb_prop]=1
            for pos in range(nb_prop):
                item = liste_item_shown[pos]
                if item not in shown_for_query.keys() :
                    shown_for_query[item]={}
                    shown_for_query[item][pos]=1
                else :
                    if pos not in shown_for_query[item].keys():
                        shown_for_query[item][pos]=1
                    else : 
                        shown_for_query[item][pos]+=1
        else : 
            pass
    return shown_for_query

In [17]:
def item_per_query(search_sessions):
    res ={}
    dict_len_prop={}
    for session in search_sessions:
        query = session.query 
        if query not in res.keys():
            res[query]=get_item_if_2pos(session,{},dict_len_prop)
        else :
            old_dict= res[query]
            res[query]=get_item_if_2pos(session,old_dict,dict_len_prop)
    return res,dict_len_prop
            

In [18]:
dico_item_query_for2,dict_len_prop_for2 = item_per_query(search_sessions)

In [43]:
def setting_data_nb_pos_diff(dico):
    clean_dico={}
    for query in dico.keys():
        clean_dico[query]={}
        for item in dico[query].keys():
            if len(dico[query][item].keys())>1:
                clean_dico[query][item]=dico[query][item]
    return clean_dico

In [52]:
def setting_data_time_pos_put(dico):
    clean_dico={}
    for query in dico.keys():
        clean_dico[query]={}
        for item in dico[query].keys():
            clean_put_pos ={}
            for pos in dico[query][item].keys():
                clean_dico[query][item]={}
                if dico[query][item][pos]>=2:
                    clean_dico[query][item][pos]=dico[query][item][int(pos)]
    return clean_dico

In [44]:
cleaned_dico = setting_data(dico_item_query_for2)
for query in cleaned_dico.keys():
    print(query)
    print('diff nb item : ',len(dico_item_query_for2[query])-len(cleaned_dico[query]))
    print ('remaining items : ',len(cleaned_dico[query]))

0
diff nb item :  149588
remaining items :  402
1
diff nb item :  81904
remaining items :  157
2
diff nb item :  42807
remaining items :  47
3
diff nb item :  25777
remaining items :  28
4
diff nb item :  15209
remaining items :  6
5
diff nb item :  9574
remaining items :  2
6
diff nb item :  6368
remaining items :  1
7
diff nb item :  4641
remaining items :  0
8
diff nb item :  2900
remaining items :  1
9
diff nb item :  1973
remaining items :  0
10
diff nb item :  1492
remaining items :  0
11
diff nb item :  1062
remaining items :  1
12
diff nb item :  822
remaining items :  0
13
diff nb item :  597
remaining items :  0
14
diff nb item :  425
remaining items :  0
15
diff nb item :  355
remaining items :  0
16
diff nb item :  297
remaining items :  0
17
diff nb item :  276
remaining items :  0
18
diff nb item :  219
remaining items :  0
19
diff nb item :  183
remaining items :  0
20
diff nb item :  120
remaining items :  0
21
diff nb item :  86
remaining items :  0
22
diff nb item :  

In [20]:
dico_item_query_for2['0']

{'34175267': {0: 174},
 '3279130': {1: 1075},
 '34171511': {0: 28},
 '3279018': {1: 28},
 '35444452': {0: 53},
 '3339757': {1: 76},
 '15370141': {0: 28},
 '1583351': {1: 28},
 '31342884': {0: 29},
 '3075887': {1: 35},
 '43630531': {0: 137},
 '3823198': {1: 147},
 '26065978': {0: 138},
 '2597528': {1: 3500},
 '29902424': {0: 52},
 '2902450': {1: 53},
 '39016998': {0: 53},
 '3549467': {1: 119},
 '62861215': {0: 12},
 '4825939': {1: 16},
 '2947180': {0: 1},
 '4807111': {1: 37},
 '44298735': {0: 1},
 '3856435': {1: 62},
 '41815016': {0: 1},
 '3712040': {1: 123},
 '62677540': {0: 1},
 '4816463': {1: 68},
 '13753389': {0: 1},
 '1410278': {1: 29},
 '3336907': {0: 1},
 '452020': {1: 1},
 '67724115': {0: 1},
 '5086282': {1: 34},
 '22354391': {0: 1},
 '2254454': {1: 439},
 '4606079': {0: 1},
 '608280': {1: 1},
 '37985498': {0: 1},
 '3486878': {1: 3},
 '53161116': {0: 1},
 '4334277': {1: 1},
 '39399823': {0: 1},
 '3562233': {1: 182},
 '39305442': {0: 1},
 '3562125': {1: 99},
 '22429663': {0: 1},


In [22]:
sorted(dict_len_prop_for2.items(), key=lambda t: t[1])

[(1, 4461), (2, 251975)]

In [95]:
dico_item_query_for2

{'0': {'34175267': {0: 242},
  '3279130': {1: 1893},
  '3140263': {0: 22, 4: 2, 5: 5, 6: 1, 1: 15, 2: 4, 3: 4},
  '2771769': {1: 48, 2: 20, 3: 1, 0: 3},
  '3809197': {2: 48, 3: 20, 1: 3},
  '34171511': {0: 36},
  '3279018': {1: 36},
  '35444452': {0: 71},
  '3339757': {1: 124},
  '15370141': {0: 36},
  '1583351': {1: 36},
  '31342884': {0: 38},
  '3075887': {1: 52},
  '43630531': {0: 184},
  '3823198': {1: 203},
  '26065978': {0: 191},
  '2597528': {1: 4476},
  '29902424': {0: 70},
  '2902450': {1: 73},
  '39016998': {0: 72},
  '3549467': {1: 171},
  '62861215': {0: 16},
  '4825939': {1: 26},
  '2947180': {0: 1},
  '4807111': {1: 154},
  '44298735': {0: 2},
  '3856435': {1: 79},
  '41815016': {0: 2},
  '3712040': {1: 547},
  '62677540': {0: 2},
  '4816463': {1: 260},
  '13753389': {0: 2},
  '1410278': {1: 75},
  '3336907': {0: 2},
  '452020': {1: 6},
  '67724115': {0: 2},
  '5086282': {1: 164},
  '22354391': {0: 2},
  '2254454': {1: 613},
  '4606079': {0: 2},
  '608280': {1: 3},
  '379

In [83]:
dico_item_query,dict_len_prop = item_per_query(search_sessions)

In [85]:
sorted(dict_len_prop.items(), key=lambda t: t[1])

[(22, 1),
 (30, 1),
 (26, 1),
 (40, 1),
 (29, 2),
 (25, 2),
 (27, 2),
 (23, 3),
 (21, 3),
 (18, 6),
 (19, 10),
 (20, 10),
 (17, 12),
 (15, 14),
 (16, 14),
 (14, 17),
 (12, 25),
 (13, 32),
 (11, 68),
 (10, 109),
 (9, 184),
 (8, 361),
 (7, 662),
 (6, 1309),
 (5, 2294),
 (4, 3876),
 (1, 4461),
 (3, 5310),
 (2, 251975)]

In [68]:
dico_item_query['0']['3140263']

{0: 18, 4: 2, 5: 5, 6: 1, 1: 5, 2: 4, 3: 4}

In [17]:
search_sessions[:3]

[{"query": "0", "task": "34573630", "web_results": [{"click": 0, "id": "3140263,2771769,3809197"}, {"click": 0, "id": "34175267,3279130"}, {"click": 0, "id": "34171511,3279018"}, {"click": 0, "id": "35444452,3339757"}, {"click": 0, "id": "15370141,1583351"}, {"click": 0, "id": "31342884,3075887"}, {"click": 0, "id": "43630531,3823198"}, {"click": 0, "id": "26065978,2597528"}, {"click": 0, "id": "29902424,2902450"}, {"click": 0, "id": "39016998,3549467"}, {"click": 0, "id": "62861215,4825939"}]},
 {"query": "0", "task": "34573635", "web_results": [{"click": 0, "id": "2947180,4807111"}, {"click": 0, "id": "44298735,3856435"}, {"click": 0, "id": "41815016,3712040"}, {"click": 0, "id": "62677540,4816463"}, {"click": 0, "id": "13753389,1410278"}, {"click": 0, "id": "3336907,452020"}, {"click": 0, "id": "67724115,5086282"}, {"click": 0, "id": "22354391,2254454"}, {"click": 0, "id": "4606079,608280"}, {"click": 0, "id": "37985498,3486878"}, {"click": 0, "id": "53161116,4334277"}]},
 {"query":

In [5]:

train_test_split = int(len(search_sessions) * 1)
train_sessions = search_sessions[:train_test_split]
train_queries = Utils.get_unique_queries(search_sessions)

test_sessions = Utils.filter_sessions(search_sessions[train_test_split:], train_queries)
test_queries = Utils.get_unique_queries(test_sessions)

print ("-------------------------------")
print ("Training on %d search sessions (%d unique queries)." % (len(train_sessions), len(train_queries)))
print ("-------------------------------")

start = time.time()
click_model.train(train_sessions)
end = time.time()
print ("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

print ("-------------------------------")
print ("Testing on %d search sessions (%d unique queries)." % (len(test_sessions), len(test_queries)))
print ("-------------------------------")

loglikelihood = LogLikelihood()
perplexity = Perplexity()

start = time.time()
ll_value = loglikelihood.evaluate(click_model, test_sessions)
end = time.time()
print ("\tlog-likelihood: %f; time: %i secs") % (ll_value, end - start)

start = time.time()
perp_value = perplexity.evaluate(click_model, test_sessions)[0]
end = time.time()
print ("\tperplexity: %f; time: %i secs") % (perp_value, end - start)

-------------------------------
Training on 24615 search sessions (42 unique queries).
-------------------------------
	Trained PBM click model in 177 secs
-------------------------------
Testing on 0 search sessions (0 unique queries).
-------------------------------


ZeroDivisionError: division by zero

In [6]:
print(search_sessions[1])
print(type(search_sessions[2]))

{"query": "0", "task": "34573635", "web_results": [{"click": 0, "id": "2947180,4807111"}, {"click": 0, "id": "44298735,3856435"}, {"click": 0, "id": "41815016,3712040"}, {"click": 0, "id": "62677540,4816463"}, {"click": 0, "id": "13753389,1410278"}, {"click": 0, "id": "3336907,452020"}, {"click": 0, "id": "67724115,5086282"}, {"click": 0, "id": "22354391,2254454"}, {"click": 0, "id": "4606079,608280"}, {"click": 0, "id": "37985498,3486878"}, {"click": 0, "id": "53161116,4334277"}]}
<class 'pyclick.click_models.task_centric.TaskCentricSearchSession.TaskCentricSearchSession'>


In [7]:
for param_name, param in click_model.params.items():
    print(param_name)
    if param_name == 'PBMParamNames.attr':
        print(len(param._container['0']))
    print('all_container',len(param._container))

PBMParamNames.attr
all_container 42
PBMParamNames.exam
all_container 10


In [50]:
PBM().params.items()

dict_items([(<PBMParamNames.attr: 1>, ), (<PBMParamNames.exam: 2>, 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000
)])

Estimation des parametres

In [53]:
click_model.params.items()

dict_items([(<PBMParamNames.attr: 1>, 0: {'3140263,2771769,3809197': 0.5000, '34175267,3279130': 0.5000, '34171511,3279018': 0.5000, '35444452,3339757': 0.5000, '15370141,1583351': 0.5000, '31342884,3075887': 0.5000, '43630531,3823198': 0.5000, '26065978,2597528': 0.5000, '29902424,2902450': 0.5000, '39016998,3549467': 0.5000, '62861215,4825939': 0.5000}
), (<PBMParamNames.exam: 2>, 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000
)])

In [51]:
print(click_model.params.items())

dict_items([(<PBMParamNames.attr: 1>, 0: {'3140263,2771769,3809197': 0.5000, '34175267,3279130': 0.5000, '34171511,3279018': 0.5000, '35444452,3339757': 0.5000, '15370141,1583351': 0.5000, '31342884,3075887': 0.5000, '43630531,3823198': 0.5000, '26065978,2597528': 0.5000, '29902424,2902450': 0.5000, '39016998,3549467': 0.5000, '62861215,4825939': 0.5000}
), (<PBMParamNames.exam: 2>, 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000
)])


In [ ]:
dict_items([(<PBMParamNames.attr: 1>, 0: {'3140263,2771769,3809197': 0.5000, '34175267,3279130': 0.5000, '34171511,3279018': 0.5000, '35444452,3339757': 0.5000, '15370141,1583351': 0.5000, '31342884,3075887': 0.5000, '43630531,3823198': 0.5000, '26065978,2597528': 0.5000, '29902424,2902450': 0.5000, '39016998,3549467': 0.5000, '62861215,4825939': 0.5000}
), (<PBMParamNames.exam: 2>, 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000 0.5000
)])

In [9]:
for param_name, param_container in self.params.items():
    param = param_container.get_for_session_at_rank(search_session, rank)
    param_dict[param_name] = param

In [10]:
param_dict

{}

In [36]:
for search_session in search_sessions[:4]:
    for rank, result in enumerate(search_session.web_results):
            param_dict = {}
    print (search_session)
    print (param_dict)

{"query": "0", "task": "34573630", "web_results": [{"click": 0, "id": "3140263,2771769,3809197"}, {"click": 0, "id": "34175267,3279130"}, {"click": 0, "id": "34171511,3279018"}, {"click": 0, "id": "35444452,3339757"}, {"click": 0, "id": "15370141,1583351"}, {"click": 0, "id": "31342884,3075887"}, {"click": 0, "id": "43630531,3823198"}, {"click": 0, "id": "26065978,2597528"}, {"click": 0, "id": "29902424,2902450"}, {"click": 0, "id": "39016998,3549467"}, {"click": 0, "id": "62861215,4825939"}]}
{}
{"query": "0", "task": "34573635", "web_results": [{"click": 0, "id": "2947180,4807111"}, {"click": 0, "id": "44298735,3856435"}, {"click": 0, "id": "41815016,3712040"}, {"click": 0, "id": "62677540,4816463"}, {"click": 0, "id": "13753389,1410278"}, {"click": 0, "id": "3336907,452020"}, {"click": 0, "id": "67724115,5086282"}, {"click": 0, "id": "22354391,2254454"}, {"click": 0, "id": "4606079,608280"}, {"click": 0, "id": "37985498,3486878"}, {"click": 0, "id": "53161116,4334277"}]}
{}
{"query"

In [62]:
search_sessions = YandexRelPredChallengeParser().parse('PyClick-master/PyClick-master/examples/data/YandexPersonalizedChallenge', 50000)

In [70]:
len(search_sessions)

24615

In [63]:
nb_click=0
nb_sessionwithclick=0
for session in search_sessions[:10]:
    if session.get_last_click_rank()<11:
        print (session.query)
        nb_sessionwithclick+=1
    nb_click+=sum(session.get_clicks())

In [64]:
print ('nb_click',nb_click)
print('nb_sessionwithclick',nb_sessionwithclick)

nb_click 0
nb_sessionwithclick 0
